<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li></ul></div>

# Introduction

This project is focused on building a scraper bot that regularly scans all listings on Immobilienscout24 and applies to those that fit the predefined criteria.

Acceptance criteria:
- The bot can open a webbrowers, login with an account and scan listings
- The bot can identify listings of interest and apply to them using an editable message template
- A history of all applied to apartments should be viewable
- Does not to send applications to apartments that were already applied to
- Listing scan frequency should be adjustable


Bonus:
- Email alert or sms is sent asking user if the apartment should be applied to
- User can answer "yes" or "no"

In [248]:
# imports

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random

import datetime as datetime
import re

import pandas as pd



In [250]:
#### testing loop

from time import sleep

import undetected_chromedriver as uc

driver = uc.Chrome()


while True:

    url = "https://sso.immobilienscout24.de/sso/login?appName=is24main&source=meinkontodropdown-login&sso_return=https://www.immobilienscout24.de/sso/login.go?source%3Dmeinkontodropdown-login%26returnUrl%3D/geschlossenerbereich/start.html?source%253Dmeinkontodropdown-login"
    driver.get(url)


    sleep(6)

    try:
        shadow_outer = expand_shadow_element(driver.find_element(By.CSS_SELECTOR, "div#usercentrics-root"))
        accept_cookie = shadow_outer.find_element(By.CSS_SELECTOR, "button[data-testid='uc-accept-all-button']")
        accept_cookie.click()
    except Exception as e:
        print(e)
        pass

    sleep_randomly()


    apply()
    
    sleep(60*5)

already applied - skipping
already applied - skipping
already applied - skipping
already applied - skipping
already applied - skipping
already applied - skipping


KeyboardInterrupt: 

In [ ]:
# WORKS!!
def expand_shadow_element(element):
    shadow_root = driver.execute_script('return arguments[0].shadowRoot', element)
    return shadow_root

In [241]:
def sleep_randomly():
    amount = random.randint(2,5)

In [251]:
def apply():
    
    # read applied history
    history_df = pd.read_csv("applied_history.csv")

    
    # sign in - user
    user_name = driver.find_element(by=By.ID, value="username")
    user_name.send_keys("moe.joe4411@gmail.com")
    sleep_randomly()

    user_name.send_keys(Keys.RETURN)

    # sign in - password
    sleep_randomly()
    password = driver.find_element(by=By.ID, value="password")
    password.send_keys("My_apartment_1")
    password.send_keys(Keys.RETURN)

    driver.get("https://www.immobilienscout24.de/Suche/shape/wohnung-mieten?shape=b3ZwX0l9aX1vQX58QW9wQWR3Qmt0Snhbb2BHaGBBc3pJc0FjfkVvekF9aUBlZkRpYUBlb0Z_Y0N9YEB2X0N5fkJoaEN8YEBsZEV8bUBuakNgakBueUN4YEJ4eUU.&haspromotion=false&numberofrooms=3.0-&price=-1350.0&exclusioncriteria=swapflat&pricetype=rentpermonth&enteredFrom=saved_search")
    sleep_randomly()


    # get all listing Id's

    def get_new_listings():
        
        result_list = driver.find_element(by=By.ID, value='listings')

        all_listings = result_list.find_elements(By.TAG_NAME, "li")    
        new_listings = [listing for listing in all_listings]

        return new_listings

    #check if listing was already seen/ applied to

    new_listings = get_new_listings()

    listing_ids = [li.get_attribute("data-id") for li in new_listings if li.get_attribute("data-id") !=None]
    
    #keep track of how many applications during a session
    applied_in_this_session = 0
    
    for li_id in listing_ids:
        
        # check if already applied
        if already_applied(history_df, li_id):
            print("already applied - skipping")
            continue
            
        # go to expose
        expose_url = f'https://www.immobilienscout24.de/expose/{li_id}'


        driver.get(expose_url)
        # click on contact
        
        # default values
        
        try:
            price = driver.find_element(by=By.XPATH, value='//*[@id="is24-content"]/div[2]/div[1]/div[2]/div[1]/div/div[1]').text
            location = driver.find_element(by=By.CLASS_NAME, value="address-block").text
            contact_name = driver.find_element(by=By.XPATH, value='//*[@id="is24-expose-contact-box"]/div/div/div[2]/div[1]/div/div[1]').text
            number_of_rooms = driver.find_element(by=By.CSS_SELECTOR, value='#is24-content > div.grid-item.padding-desk-right-xl.desk-two-thirds.lap-one-whole.desk-column-left.flex-item.palm--flex__order--1.lap--flex__order--1 > div.is24-ex-details.main-criteria-headline-size.two-column-layout > div.criteriagroup.flex.flex--wrap.main-criteria-container > div:nth-child(2) > div.is24qa-zi-main.is24-value.font-semibold').text
            size = driver.find_element(by=By.CSS_SELECTOR, value='#is24-content > div.grid-item.padding-desk-right-xl.desk-two-thirds.lap-one-whole.desk-column-left.flex-item.palm--flex__order--1.lap--flex__order--1 > div.is24-ex-details.main-criteria-headline-size.two-column-layout > div.criteriagroup.flex.flex--wrap.main-criteria-container > div:nth-child(3) > div.is24qa-flaeche-main.is24-value.font-semibold').text
            contact = driver.find_element(by=By.ID, value="contact-realtor-desk").click()
            sleep(4)
            
       
            message_text = driver.find_element(by=By.ID, value="contactForm-Message")

            message_text.send_keys(get_message(contact_name=contact_name,
                                               num_rooms=number_of_rooms,
                                               location=location))
            sleep_randomly()
            sleep(10)



            # Click on send profile

            # Click apply


            # add listing to csv
            apply_date = str(datetime.datetime.now().date())

            add_to_history(li_id, location, number_of_rooms, price, size, contact_name, expose_url, apply_date)

            applied_in_this_session +=1
        
        except Exception as e:
            print(f"ERROR: {e}/n\n Skipping this expose...")
            continue

        
    driver.close()
    return applied_in_this_session






In [243]:
def get_message(contact_name=None, num_rooms="3", location=""):

    title = None         
    if "herr" in contact_name.lower():
        last_name = contact_name.split()[-1]
        greetings = f"Hallo Herr {last_name},"


    elif "frau" in contact_name.lower():
        last_name = contact_name.split()[-1]
        greetings = f"Hallo Frau {last_name},"

    else:
        greetings = f"Hallo,"

    plz = int(re.findall(r"\D(\d{5})\D", " "+location+" ")[0])
    district = get_districs(plz)

    message_template = f"""{greetings}

wir sind ein Paar aus Berlin und suchen nun eine größere Wohnung, um unser Leben als Ehepaar zu beginnen. Wir sind beide 28 Jahre alt, berufstätig und haben ein Gesamteinkommen von 5000 EUR netto pro Monat. Ihre schöne {num_rooms}-Zimmer-Wohnung in {district} ist uns sehr aufgefallen und wir würden uns über einen Besichtigungstermin sehr freuen. Vielen Dank im Voraus.

Mit freundlichen Grüßen
Moe und Allegra
    """

    return message_template

In [244]:
def add_to_history(*args):
    
    data_dict = {"ID": [int(args[0])],
             "Location": [args[1].replace("\n"," ")],
             "Number_of_rooms": [args[2]],
             "Kaltmiete" : [args[3]],
             "Size": [args[4]],
             "Contact_name": [args[5]],
             "Link": [args[6]],
             "Date":[args[7]]}
    
    df_new = pd.DataFrame(data_dict)
    df = pd.read_csv("applied_history.csv", index_col=None)
    
    df_updated = pd.concat([df, df_new], axis=0)
    
    df_updated.to_csv("applied_history.csv", index=False, encoding="utf-8")
    
    return print(args)

In [245]:
def already_applied(history_df, listing_id):
    
    if int(listing_id) in history_df["ID"].values:
        
        return True
    
    return False

In [246]:
def get_districs(plz):
    
    df_plz = pd.read_excel("berlin_plz.xlsx", index_col="PLZ")
    
    if plz in df_plz.index:
        return df_plz.loc[plz]["District"]
    return None

In [247]:
data_dict = {"ID": [1234],
             "Location": ["test"],
             "Number_of_rooms": ["test"],
             "Kaltmiete" : ["test"],
             "Size": ["test"],
             "Contact_name": ["test"],
             "Link": ["test"],
             "Date":["test"]}


applications_data = pd.DataFrame(data_dict)

applications_data.to_csv("applied_history.csv", index=None)